In [20]:
import pandas as pd
import re
import random

# ASTE

In [122]:
#train
train = pd.read_csv('D:/Kuliah/S2/Tesis/Code/Generative-ABSA-main/data/aste/rest15/train.txt', sep='.####', header=None, names=['input', 'label'])
train['label'] = train['label'].apply(lambda x:x[1:-1])
#test
test = pd.read_csv('D:/Kuliah/S2/Tesis/Code/Generative-ABSA-main/data/aste/rest15/test.txt', sep='.####', header=None, names=['input', 'label'])
test['label'] = test['label'].apply(lambda x:x[1:-1])
#dev
dev = pd.read_csv('D:/Kuliah/S2/Tesis/Code/Generative-ABSA-main/data/aste/rest15/dev.txt', sep='.####', header=None, names=['input', 'label'])
dev['label'] = dev['label'].apply(lambda x:x[1:-1])
dev.head()

c:\Users\danendra\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,input,label
0,The food is very average ... the Thai fusion s...,"([1], [4], 'NEG'), ([7, 8, 9], [13, 14], 'NEG')"
1,The decor is night tho ... but they REALLY nee...,"([29], [31], 'NEG'), ([29], [33], 'NEG'), ([1]..."
2,The spicy tuna roll was unusually good and the...,"([1, 2, 3], [6], 'POS'), ([9, 10, 11], [13], '..."
3,we love th pink pony,"([3, 4], [1], 'POS')"
4,THe perfect spot,"([2], [1], 'POS')"


In [123]:
train.shape[0]+test.shape[0]+dev.shape[0]

1075

In [105]:
def convert_int(duplet):
    for d in range(len(duplet)):
        for i in range(len(duplet[d])):
            duplet[d][i]=int(duplet[d][i])
    return duplet

In [106]:
def convert_duplet(out):
    #untuk 1 output
    all_triplet = out.split("),")
    #find all in closed bracket
    duplet = [re.findall('\[(.*?)\]', triplet) for triplet in all_triplet]
    duplet = [[i.split(',') for i in d] for d in duplet]
    duplet = [convert_int(d) for d in duplet]
    #find all sentiment
    sentiment = [re.findall("\'.*?\'", triplet) for triplet in all_triplet]
    sentiment = [sent[0][1:-1] for sent in sentiment]
    return duplet, sentiment

In [107]:
#ubah angka ke kata
def convert_int_string(inp, duplet):
    inp_list = inp.split()
    for i in range(len(duplet)):
        for j in range(len(duplet[i])):
            for k in range(len(duplet[i][j])):
                duplet[i][j][k] = inp_list[duplet[i][j][k]]
    return duplet

In [108]:
def convert_triplet(inp, out):
    duplet, sentiment = convert_duplet(out)
    duplet = convert_int_string(inp, duplet)
    label = [duplet[i]+[sentiment[i]] for i in range(len(duplet))]
    #all triplet in triplets
    triplet_str_arr = []
    for i,triplet in enumerate(label):
        triplet_str = ""
        #get all tuple
        for j,tuple in enumerate(triplet):
            #get all aspect from the same tuple
            if type(tuple)==str:
                triplet_str+=tuple
            else:
                triplet_str+=" ".join(tuple)
            if j<(len(triplet)-1):
                triplet_str+=','
        triplet_str=f'({triplet_str})'
        if i<(len(label)-1):
            triplet_str+=';'
        triplet_str_arr.append(triplet_str)
    label = "".join(triplet_str_arr)
    return label

In [109]:
def transform_label(df, x, y):
    labels = []
    errors = []
    for i in range(len(df)):
        try:
            row = df.iloc[i]
            inp = row[x]
            out = row[y]
            triplet = convert_triplet(inp, out)
            labels.append(triplet)
        except:
            errors.append(i)
    return labels, errors

In [110]:
y_train, train_errors = transform_label(train, 'input', 'label')
y_test, test_errors = transform_label(test, 'input', 'label')
y_dev, dev_errors = transform_label(dev, 'input', 'label')

In [111]:
train = train.drop(train_errors)
test = test.drop(test_errors)
dev = dev.drop(dev_errors)

In [112]:
train['triplet'] = y_train
test['triplet'] = y_test
dev['triplet'] = y_dev

In [113]:
pd.concat([train, test, dev]).to_csv('D:/Kuliah/S2/Tesis/Code/Generative-ABSA-main/data/aste/rest15/all_data.csv', index=False)

# TASD

In [124]:
#train
train = pd.read_csv('D:/Kuliah/S2/Tesis/Code/Generative-ABSA-main/data/tasd/rest15/train.txt', sep='.####', header=None, names=['input', 'label'])
train['label'] = train['label'].apply(lambda x:x[1:-1])
#test
test = pd.read_csv('D:/Kuliah/S2/Tesis/Code/Generative-ABSA-main/data/tasd/rest15/test.txt', sep='.####', header=None, names=['input', 'label'])
test['label'] = test['label'].apply(lambda x:x[1:-1])
#dev
dev = pd.read_csv('D:/Kuliah/S2/Tesis/Code/Generative-ABSA-main/data/tasd/rest15/dev.txt', sep='.####', header=None, names=['input', 'label'])
dev['label'] = dev['label'].apply(lambda x:x[1:-1])
dev.head()

c:\Users\danendra\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,input,label
0,Judging from previous posts this used to be a ...,"('place', 'restaurant general', 'negative')"
1,"We , there were four of us , arrived at noon -...","('staff', 'service general', 'negative')"
2,"They never brought us complimentary noodles , ...","('NULL', 'service general', 'negative')"
3,The food was lousy - too sweet or too salty an...,"('food', 'food quality', 'negative'), ('portio..."
4,"After all that , they complained to me about t...","('NULL', 'service general', 'negative')"


In [128]:
tasd_df = pd.concat([train, test, dev])
tasd_df

,input,label
0,Judging from previous posts this used to be a ...,"('place', 'restaurant general', 'negative')"
1,"We , there were four of us , arrived at noon -...","('staff', 'service general', 'negative')"
2,"They never brought us complimentary noodles , ...","('NULL', 'service general', 'negative')"
3,The food was lousy - too sweet or too salty an...,"('food', 'food quality', 'negative'), ('portio..."
4,"After all that , they complained to me about t...","('NULL', 'service general', 'negative')"
...,...,...
5,Avoid this place,"('place', 'restaurant general', 'negative')"
6,"I have eaten at Saul , many times , the food i...","('food', 'food quality', 'positive')"
7,Saul is the best restaurant on Smith Street an...,"('Saul', 'restaurant general', 'positive')"
8,The duck confit is always amazing and the foie...,"('foie gras terrine with figs', 'food quality'..."


In [159]:
#aste data
aste_df = pd.read_csv('D:/Kuliah/S2/Tesis/Code/Generative-ABSA-main/data/aste/rest15/all_data.csv')
aste_df = aste_df.rename(columns={'label':'aste_label'})
aste_df

,input,aste_label,triplet
0,Judging from previous posts this used to be a ...,"([10], [9], 'NEG')","(place,good,NEG)"
1,"We , there were four of us , arrived at noon -...","([19], [31], 'NEG')","(staff,rude,NEG)"
2,The food was lousy - too sweet or too salty an...,"([1], [3], 'NEG'), ([1], [5, 6], 'NEG'), ([1],...","(food,lousy,NEG);(food,too sweet,NEG);(food,to..."
3,Avoid this place,"([2], [0], 'NEG')","(place,Avoid,NEG)"
4,"I have eaten at Saul , many times , the food i...","([10], [16], 'POS')","(food,good,POS)"
...,...,...,...
1070,The food is okay and the prices here are medio...,"([1], [3], 'NEU')","(food,okay,NEU)"
1071,Baluchi 's has solid food and a nice decor at ...,"([4], [3], 'POS'), ([8], [7], 'POS'), ([0, 1],...","(food,solid,POS);(decor,nice,POS);(Baluchi 's,..."
1072,$ 20 for all you can eat sushi can not be beaten,"([3, 4, 5, 6, 7], [11], 'POS')","(all you can eat sushi,beaten,POS)"
1073,"The vibe is very relaxed and cozy , service wa...","([1], [4], 'POS'), ([1], [6], 'POS'), ([8], [1...","(vibe,relaxed,POS);(vibe,cozy,POS);(service,gr..."


In [157]:
aste_df.shape

(1075, 3)

In [160]:
merged_df = aste_df.join(tasd_df.set_index('input'), on='input')
merged_df = merged_df[~merged_df.index.duplicated(keep='first')]
merged_df = merged_df.dropna()
merged_df

,input,aste_label,triplet,label
0,Judging from previous posts this used to be a ...,"([10], [9], 'NEG')","(place,good,NEG)","('place', 'restaurant general', 'negative')"
1,"We , there were four of us , arrived at noon -...","([19], [31], 'NEG')","(staff,rude,NEG)","('staff', 'service general', 'negative')"
2,The food was lousy - too sweet or too salty an...,"([1], [3], 'NEG'), ([1], [5, 6], 'NEG'), ([1],...","(food,lousy,NEG);(food,too sweet,NEG);(food,to...","('food', 'food quality', 'negative'), ('portio..."
3,Avoid this place,"([2], [0], 'NEG')","(place,Avoid,NEG)","('place', 'restaurant general', 'negative')"
4,"I have eaten at Saul , many times , the food i...","([10], [16], 'POS')","(food,good,POS)","('food', 'food quality', 'positive')"
...,...,...,...,...
1068,"whoever the jazz duo was , they were on POINT","([2, 3], [8, 9], 'POS')","(jazz duo,on POINT,POS)","('jazz duo', 'ambience general', 'positive')"
1069,even the wine by the glass was good,"([2, 3, 4, 5], [7], 'POS')","(wine by the glass,good,POS)","('wine by the glass', 'drinks quality', 'posit..."
1070,The food is okay and the prices here are medio...,"([1], [3], 'NEU')","(food,okay,NEU)","('food', 'food quality', 'neutral'), ('NULL', ..."
1071,Baluchi 's has solid food and a nice decor at ...,"([4], [3], 'POS'), ([8], [7], 'POS'), ([0, 1],...","(food,solid,POS);(decor,nice,POS);(Baluchi 's,...","('food', 'food quality', 'positive'), ('decor'..."


In [181]:
idx = random.randint(0, len(merged_df))
row = merged_df.iloc[idx]
inp = row['input']
triplet = row['triplet']
aste_label = row['aste_label']
label = row['label']
print(inp)
print(aste_label)
print(triplet)
print(label)
all_categories = label.split(')')
#all_categories

Add to that great service and great food at a reasonable price and you have yourself the beginning of a great evening 
([4], [3], 'POS'), ([7], [6], 'POS')
(service,great,POS);(food,great,POS)
('service', 'service general', 'positive'), ('food', 'food quality', 'positive'), ('food', 'food prices', 'positive')


In [203]:
def extract_pairs(label):
    """
    get all pairs from TASD label, the pair is
    (aspect, aspect category)
    """
    all_tasd = re.findall('\[(.*?)\]', label.replace('(','[').replace(')',']').replace("'",''))
    all_tasd = [tasd.split(',') for tasd in all_tasd]
    all_pair = [(tasd[0].strip(), tasd[1].strip()) for tasd in all_tasd]
    return all_pair

In [268]:
def convert_to_quadruplet(pairs, triplets):
    triplets = triplets.split(';')
    triplets = [triplet[1:-1].split(',') for triplet in triplets]
    quadruplets = []
    #triplet_list[0] is aspect
    for triplet in triplets:
        aspect = triplet[0]
        quadruplet=[]
        for pair in pairs:
            #if aspect in pair from TASD is the same as 
            #aspect in triplet from ASTE then combine it
            if pair[0]==aspect:
                quadruplet = triplet+[pair[1]]
        #if there are no same aspect from TASD in ASTE that means
        #we will drop the triplet
        if len(quadruplet)>=4:
            quadruplets.append(quadruplet)
    return quadruplets
        

In [260]:
def string_quadruplet(quadruplets):
    quadruplets_str = ''
    for i,quadruplet in enumerate(quadruplets):
        quadruplet_str = f'({str(quadruplet)})'
        quadruplets_str+=quadruplet_str.replace('[','').replace(']','').replace("'",'')
        if i<(len(quadruplets)-1):
            quadruplets_str+=';'
    return quadruplets_str

In [273]:
#idx = random.randint(0, len(merged_df))
idx=305
row = merged_df.iloc[idx]
inp = row['input']
triplet = row['triplet']
aste_label = row['aste_label']
label = row['label']
print(inp)
print(triplet)
print(label)
#pair
all_pair = extract_pairs(label)
#quadruplet
quadruplet = convert_to_quadruplet(all_pair, triplet)
quadruplet = string_quadruplet(quadruplet)
print(quadruplet)

An excellent servic
(servic,excellent,POS)
('service', 'service general', 'positive')



In [285]:
def transform_to_quadruplet(df, triplet_col, tasd_col):
    quadruplets = []
    for i in range(len(df)):
        row = df.iloc[i]
        triplet = row[triplet_col]
        label = row[tasd_col]
        #pair
        all_pair = extract_pairs(label)
        #quadruplet
        quadruplet = convert_to_quadruplet(all_pair, triplet)
        quadruplet = string_quadruplet(quadruplet)
        quadruplets.append(quadruplet)
    return quadruplets

In [287]:
y = transform_to_quadruplet(merged_df, 'triplet', 'label')
y

['(place, good, NEG, restaurant general)',
 '(staff, rude, NEG, service general)',
 '(food, lousy, NEG, food quality);(food, too sweet, NEG, food quality);(food, too salty, NEG, food quality);(portions, tiny, NEG, food style_options)',
 '(place, Avoid, NEG, restaurant general)',
 '(food, good, POS, food quality)',
 '(Saul, best, POS, restaurant general)',
 '(foie gras terrine with figs, out of this world, POS, food quality);(duck confit, amazing, POS, food quality)',
 '(wine list, interesting, POS, drinks prices);(wine list, good values, POS, drinks prices)',
 '(restaurant, disappointed, NEG, restaurant general)',
 '(Food, okay, NEU, food quality);(Food, nothing great, NEU, food quality)',
 '(service, excellent, POS, service general);(decor, cool, POS, ambience general);(decor, understated, POS, ambience general)',
 '(duck breast special, incredible, POS, food quality)',
 '(Grilled Chicken special with Edamame Puree, enjoyed, POS, food quality)',
 '(food, decent, NEU, food quality)',
 

In [288]:
quadruplet_df = merged_df.drop(columns=['aste_label'])
quadruplet_df['quadruplet_label'] = y


In [293]:
quadruplet_df[quadruplet_df['quadruplet_label']!=''].to_csv('D:/Kuliah/S2/Tesis/Code/Generative-ABSA-main/data/quadruplet/rest15/all_data.csv', index=False)

In [292]:
quadruplet_df

,input,triplet,label,quadruplet_label
0,Judging from previous posts this used to be a ...,"(place,good,NEG)","('place', 'restaurant general', 'negative')","(place, good, NEG, restaurant general)"
1,"We , there were four of us , arrived at noon -...","(staff,rude,NEG)","('staff', 'service general', 'negative')","(staff, rude, NEG, service general)"
2,The food was lousy - too sweet or too salty an...,"(food,lousy,NEG);(food,too sweet,NEG);(food,to...","('food', 'food quality', 'negative'), ('portio...","(food, lousy, NEG, food quality);(food, too sw..."
3,Avoid this place,"(place,Avoid,NEG)","('place', 'restaurant general', 'negative')","(place, Avoid, NEG, restaurant general)"
4,"I have eaten at Saul , many times , the food i...","(food,good,POS)","('food', 'food quality', 'positive')","(food, good, POS, food quality)"
...,...,...,...,...
1068,"whoever the jazz duo was , they were on POINT","(jazz duo,on POINT,POS)","('jazz duo', 'ambience general', 'positive')","(jazz duo, on POINT, POS, ambience general)"
1069,even the wine by the glass was good,"(wine by the glass,good,POS)","('wine by the glass', 'drinks quality', 'posit...","(wine by the glass, good, POS, drinks quality)"
1070,The food is okay and the prices here are medio...,"(food,okay,NEU)","('food', 'food quality', 'neutral'), ('NULL', ...","(food, okay, NEU, food quality)"
1071,Baluchi 's has solid food and a nice decor at ...,"(food,solid,POS);(decor,nice,POS);(Baluchi 's,...","('food', 'food quality', 'positive'), ('decor'...","(food, solid, POS, food quality);(decor, nice,..."
